<a href="https://colab.research.google.com/github/nilanshutwinkle/random-notes/blob/master/Learning_Langgraph_AgenticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LEARNING LANGGRAPH
---
Calling Tools

In [7]:
!pip install langchain langchain_openai langchain-experimental langchain-community openai langchain-core langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.6 MB/s eta 0:00:00


In [13]:
import os, sys, re, json, asyncio

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

False

In [11]:
model = ChatOpenAI(model = 'gpt-4o-mini', api_key = userdata.get('OPENAI_API_KEY'))

In [30]:
def call_llm(state: MessagesState):
  messages = state["messages"]
  response = model.invoke(messages)
  print(response)
  return {"messages": [response]}

In [31]:
workflow = StateGraph(MessagesState)

workflow.add_node('call_llm', call_llm)

workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)

app = workflow.compile(checkpointer=MemorySaver())

In [33]:
def interact_with_agent_with_true_session():
  while True:
    thread_id = input("Enter threadId, or type 'new' to start with a fresh one")
    if thread_id.lower() in ['quit', 'exit', 'bye']:
      print("Ending conversation ... ")
      break

    if thread_id.lower() == 'new':
      thread_id = f"session_{os.urandom(4).hex()}"

    while True:
      user_input = input("You: ")
      if user_input.lower() in ['quit', 'bye', 'exit']:
        print(f'Ending Session {thread_id}')
        break

      input_message = {
      "messages": [("human", user_input)]
      }
      # Invoke the graph with the correct thread ID to maintain memory acrosssessions
      config = {"configurable": {"thread_id": thread_id}}

      for chunk in app.stream(input_message, config=config, stream_mode="values"):
        chunk["messages"][-1].pretty_print()

interact_with_agent_with_true_session()

Enter threadId, or type 'new' to start with a fresh onenew
You: input_message = { "messages": [("human", user_input)] } # Invoke the graph with the correct thread ID to maintain memory across sessions config = {"configurable": {"thread_id": thread_id}} for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"): chunk["messages"][-1].pretty_print()
================================ Human Message =================================

input_message = { "messages": [("human", user_input)] } # Invoke the graph with the correct thread ID to maintain memory across sessions config = {"configurable": {"thread_id": thread_id}} for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"): chunk["messages"][-1].pretty_print()
content='It looks like you are trying to create a system that utilizes a chat interface where messages are sent and responses are streamed back, with the use of some sort of memory configuration. Your input message contains 

## In Memory Store
---
Memory Store allows the agent to store information that can be shared
across different sessions (threads) for the same user.

In [42]:
from langgraph.store.memory import InMemoryStore
import uuid

in_memory_store = InMemoryStore()
def store_user_info(state: MessagesState, config, *, memory_store = in_memory_store):
  user_id = config["configurable"]["user_id"]
  namespace = (user_id, "memories")

  # Create a memory based on conversation
  memory_id = str(uuid.uuid4())
  memory = {"user_name": state["user_name"]}

  # Save the memory to in-memory store
  memory_store.put(namespace, memory_id, memory)
  return {"message": ["User information saved."]}

# Function to retrive stored user information
def retrive_user_info(state: MessagesState, config, *, memory_store = in_memory_store):
  user_id = config["configurable"]["user_id"]
  namespace = (user_id, "memories")

  # Retrieve the stored memory
  memories = memory_store.search(namespace)
  if memories:
    info = f'Hello {memories[-1].value["user_name"]}, welcome back!'
  else:
    info = f'No user information found'

  return {"message": [info]}

### Complete code

In [43]:
import os, sys, re, json, asyncio

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.memory import InMemoryStore
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

False

In [44]:
model = ChatOpenAI(model = 'gpt-4o-mini', api_key = userdata.get('OPENAI_API_KEY'))

In [45]:
in_memory_store = InMemoryStore()

In [46]:
def call_model(state: MessagesState, config):
  last_message = state["messages"][-1].content.lower()
  if 'remember my name is' in last_message:
    user_name = last_message[-1].split('remember my name is')[-1].strip()
    state['user_name'] = user_name
    return store_user_info(state, config)

  if 'what is my name' in last_message:
    return retrive_user_info(state, config)

  return {"messages": ["Sorry I didn't understand your request."]}

In [47]:
workflow = StateGraph(MessagesState)

workflow.add_node('call_model', call_model)

workflow.add_edge(START, 'call_model')
workflow.add_edge('call_model', END)

app = workflow.compile(checkpointer=MemorySaver(), store= in_memory_store)

In [48]:
def simulate_session():
  config = {"configurable" : {"thread_id": "session_1", "user_id": "user_123"}}

  input_message = {"messages": [{"type": "user", "content": "Remember my name is Bruno"}]}
  for chunk in app.stream(input_message, config=config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

  config = {"configurable" : {"thread_id": "session_2", "user_id": "user_123"}}
  input_message = {"messages": [{"type": "user", "content": "What is my name"}]}
  for chunk in app.stream(input_message, config=config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

simulate_session()

================================ Human Message =================================

Remember my name is Bruno
================================ Human Message =================================

What is my name
